In [1]:
using Random
using Distributions
using FLoops
using Base.Threads

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [21]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 100    # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8     # size of group 2
px, py = partition(nx, ny)
mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end

dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)

distrTypeX = LogNormal{dtype}
X_mu = dtype.(round.(rand(B), digits=3))
X_sigma = dtype.(round.(rand(B) * 0.6, digits=3))
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Gamma{dtype}
Y_shape = @. 1 / (exp(X_sigma^2) - 1)
Y_scale = @. exp(X_mu + X_sigma^2 / 2) / Y_shape
distrY = map(distrTypeY, Y_shape, Y_scale)

@show distrX[1]
@show distrY[1]

deltas = @. mean(distrX) - mean(distrY);

distrX[1] = LogNormal{Float32}(μ=0.906f0, σ=0.572f0)
distrY[1] = Gamma{Float32}(α=2.5836008f0, θ=1.1279569f0)


In [22]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], nx, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


8×1800 Matrix{Float32}:
 2.11136   2.6391    1.90156   2.00924   …  1.1069   1.53412   2.7346
 1.64843   2.95654   2.34013   3.09854      0.7289   0.64576   2.83896
 0.627449  1.19132   2.54112   0.57285      1.879    1.32214   1.07355
 2.57078   2.25439   0.436584  0.606195     1.14569  4.97861   6.19997
 4.01339   2.41301   1.89056   2.58368      2.40217  1.16623   0.868414
 2.27239   0.260851  2.46342   1.77637   …  3.36924  3.41251   0.814334
 1.41211   0.676851  1.63079   2.18224      3.02213  3.22583   4.04552
 1.37667   0.765134  1.66696   0.737053     2.08818  0.721793  0.55302

In [23]:
# flatten into 3D matrix
X = reshape(hcat(xs...), 8, S, B)
Y = reshape(hcat(ys...), 8, S, B)
Y[:,:,2]

8×1800 Matrix{Float32}:
 2.11136   2.6391    1.90156   2.00924   …  1.1069   1.53412   2.7346
 1.64843   2.95654   2.34013   3.09854      0.7289   0.64576   2.83896
 0.627449  1.19132   2.54112   0.57285      1.879    1.32214   1.07355
 2.57078   2.25439   0.436584  0.606195     1.14569  4.97861   6.19997
 4.01339   2.41301   1.89056   2.58368      2.40217  1.16623   0.868414
 2.27239   0.260851  2.46342   1.77637   …  3.36924  3.41251   0.814334
 1.41211   0.676851  1.63079   2.18224      3.02213  3.22583   4.04552
 1.37667   0.765134  1.66696   0.737053     2.08818  0.721793  0.55302

In [26]:
function save_permutation_results(results, B, S, dir="./")
    i = 1
    per_method = []
    for is_two_sided in [true, false]
        for pooled in [true, false]
            averages = []

            for row in eachrow(results[i,:,:])
                coverage = sum([j for (j, _) in row]) / S
                width = sum([j for (_, j) in row]) / S
                push!(averages, (coverage, width))
            end

            alpha_ = is_two_sided ? alpha : alpha / 2
            save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, is_two_sided, parent_dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 2 methods)

In [29]:
B = 100
S = 1800

T = Threads.nthreads()
results = Array{Union{Nothing, Tuple}, 3}(undef, 6, B, S)
fill!(results, nothing)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]

    results[1, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, 0.05, twoSided, twoSided)    
    results[2, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha, twoSided, twoSided)
    results[3, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha/2, greater, smaller)
    results[4, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha/2, greater, smaller)
    
    """
    lo, hi = bootstrap(x, y, 0.05, false, 10_000)
    coverages[5] += (lo <= delta <= hi)

    lo, hi = tconf(x, y, 0.05, false)
    coverages[6] += (lo <= delta <= hi)
    """
end

save_permutation_results(results, B, S)

1856.476325 seconds (1.12 G allocations: 12.290 TiB, 72.62% gc time, 0.01% compilation time)


In [28]:
averages = []
for row in eachrow(results[1,:,:])
    coverage = sum([j for (j, _) in row]) / S
    width = sum([j for (_, j) in row]) / S
    push!(averages, (coverage, width))
end
averages

5-element Vector{Any}:
 (0.95, 3.1827745949945476)
 (0.92, 1.6322506846556246)
 (0.85, 1.3739451044574138)
 (0.93, 0.21217739206443034)
 (0.9, 0.18690359039665994)